In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
def Low_Vol(df, numdays=60, ndays=1, window=None):
    '''
    This function will execute low vol strategy by calculating the standard deviation of past days's daily returns
    :param df: querydata
    :param numdays: look at how many days back
    :return:
    '''
    # key metric calculation and data prep
    grouped = df.groupby(df.ticker)
    # enddate = df['date'].iloc[-1]
    tickerDFlist = []
    # do calculation based on single stock
    for _, group in grouped:
        group.loc[:, 'key_metric'] = group['daily_return'].rolling(numdays).std()
        # group.loc[:, 'key_metric'] = group['daily_return'].rolling(numdays).std()
        group.loc[:, 'daily_return_shifted'] = group['daily_return'].shift(-1)
        # group.loc[:, 'daily_return_shifted'] = group['daily_return_shifted']
        group.fillna(0, inplace=True)
        for i in range(0, ndays):
            group.loc[:, f'price_shifted_{i + 1}'] = group['price'].shift(-i - 1)
        # group.fillna(0, inplace=True)
        group = group[group['insp500'] == 1]  # stock universe
        tickerDFlist.append(group)
    calcDF = pd.concat(tickerDFlist)
    ##calcDF = calcDF.dropna()??????
    # for i in range(1, ndays):
    #     calcDF = calcDF.drop([f'price_shifted_{i+1}'],axis=1)
    calcDF = calcDF.reset_index(drop=True)
    # print(calcDF["date"].iloc[-1])
    # why na value treat differently
    # low vol will pick lots of stock acquired someday, which can be known before it happens
    # if use ndays, drop the stock if does not have prices in ndays
    # not like mr will pick stock bankrupted, which cannot be know before it happens
    return calcDF

def Mean_Reversion(df, numdays=70, ndays=1, window=None, with_vol=False):
    '''
        :param df: querydata
        :param numdays: look at how many days back
        :return:
        '''
    # key metric calculation and data prep
    grouped = df.groupby(df.ticker)
    # enddate = df['date'].iloc[-1]
    tickerDFlist = []
    # do calculation based on single stock
    for _, group in tqdm(grouped):
        # group=group.reset_index()
        # group=group.reset_index()
        #group.loc[:, 'rollingmean']=group['total_return'].shift(30)
        # group['rollingmean'] = group['rollingmean'].rolling(130).mean()
        # group.loc[:, 'key_metric'] = group['total_return'] / group['rollingmean'] - 1
        #group.loc[:, 'rollingmean']=group['total_return'].rolling(numdays).mean()
        #group['rollingmean']=group['rollingmean'].shift(1)
        #group.loc[:, 'key_metric'] = group['total_return'] / group['rollingmean'] - 1

        group.loc[:,'key_metric'] = group['total_return'] / group['total_return'].shift(numdays) - 1 # original
        # group.loc[:,'key_metric'] = group['price'] / group['price'].shift(numdays) - 1
        group.dropna(inplace=True)
        group.fillna(-1, inplace=True)
        for i in range(0,ndays):
            group.loc[:,f'price_shifted_{i+1}'] = group['price'].shift(-i-1)
        group.loc[:, 'daily_return_shifted'] = group['daily_return'].shift(-1)
        group.fillna(-1, inplace=True)
        group = group[group['insp500'] == 1.0]
        tickerDFlist.append(group)
    calcDF = pd.concat(tickerDFlist)
    calcDF = calcDF.reset_index(drop=True)
    return calcDF
